In [2]:
import pandas as pd
import dask.dataframe as dd
import warnings
from shapely import wkt


import pyproj
from shapely.geometry import Point
import geopandas as gpd
from shapely.ops import transform
from functools import partial

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

/opt/homebrew/anaconda3/envs/DaskEnv/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
networkData = pd.read_csv("data/networks.csv")
networkData

,osmid,y,x,street_count,ref,highway,geometry,degree
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.9635764 40.7907204),4
1,30978747,40.774267,-73.973425,4,NaN,NaN,POINT (-73.973425 40.7742666),4
2,30978752,40.774754,-73.974383,4,NaN,NaN,POINT (-73.9743829 40.7747541),4
3,39076461,40.786409,-73.794627,3,33,motorway_junction,POINT (-73.7946273 40.7864093),3
4,39076490,40.762429,-73.757091,3,31W,motorway_junction,POINT (-73.7570906 40.7624294),3
...,...,...,...,...,...,...,...,...
345954,11853233947,40.741609,-73.861996,4,NaN,NaN,POINT (-73.861996 40.7416089),4
345955,11853233948,40.741462,-73.861929,4,NaN,NaN,POINT (-73.8619292 40.7414618),4
345956,11853233953,40.741738,-73.861778,4,NaN,NaN,POINT (-73.8617783 40.7417385),4
345957,11853233954,40.741522,-73.861816,4,NaN,crossing,POINT (-73.8618164 40.7415217),4


In [34]:
networkData.nunique()

osmid           345959
y               324690
x               327587
street_count         6
ref                186
highway             13
geometry        345959
degree               7
dtype: int64

In [ ]:
zipcodeData = pd.read_csv('data/Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv')

In [39]:
networkData['geometry'] = networkData['geometry'].apply(wkt.loads)
# zipcodeData['the_geom'] = zipcodeData['the_geom'].apply(wkt.loads)

gdf_points = gpd.GeoDataFrame(networkData, geometry='geometry')
gdf_polygons = gpd.GeoDataFrame(zipcodeData, geometry='the_geom')

# Set a common Coordinate Reference System
gdf_points.crs = gdf_polygons.crs = "EPSG:4326"
result = gpd.sjoin(gdf_points, gdf_polygons, how="inner", op='within')

In [42]:
result

,osmid,y,x,street_count,ref,highway,geometry,degree,index_right,MODZCTA,label,ZCTA,pop_est
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,177,99999,NaN,99999,0
1,30978747,40.774267,-73.973425,4,NaN,NaN,POINT (-73.97343 40.77427),4,177,99999,NaN,99999,0
2,30978752,40.774754,-73.974383,4,NaN,NaN,POINT (-73.97438 40.77475),4,177,99999,NaN,99999,0
3,39076461,40.786409,-73.794627,3,33,motorway_junction,POINT (-73.79463 40.78641),3,132,11360,11360,"11359, 11360",19621
4,39076490,40.762429,-73.757091,3,31W,motorway_junction,POINT (-73.75709 40.76243),3,135,11362,11362,11362,18721
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345954,11853233947,40.741609,-73.861996,4,NaN,NaN,POINT (-73.86200 40.74161),4,144,11368,11368,11368,112425
345955,11853233948,40.741462,-73.861929,4,NaN,NaN,POINT (-73.86193 40.74146),4,144,11368,11368,11368,112425
345956,11853233953,40.741738,-73.861778,4,NaN,NaN,POINT (-73.86178 40.74174),4,144,11368,11368,11368,112425
345957,11853233954,40.741522,-73.861816,4,NaN,crossing,POINT (-73.86182 40.74152),4,144,11368,11368,11368,112425


In [11]:
crashData = dd.read_csv("data/newFinal/0.csv")

In [13]:
crashData.head()

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity


In [15]:
def getCircle(row):
    def create_circle(latitude, longitude, radius_meters):
        proj = pyproj.Proj(proj='aeqd', ellps='WGS84', datum='WGS84', lat_0=latitude, lon_0=longitude)
        project_to_aeqd = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'), proj)
        project_to_lonlat = partial(pyproj.transform, proj, pyproj.Proj(init='epsg:4326'))
        center_point = transform(project_to_aeqd, Point(longitude, latitude))
        circle = center_point.buffer(radius_meters)
        circle_lonlat = transform(project_to_lonlat, circle)

        return circle_lonlat

    circle = create_circle(row['LATITUDE'], row['LONGITUDE'], 300)
    return circle


crashData['circlePolygon'] = crashData.apply(lambda x: getCircle(x), axis = 1)

In [16]:
crashData.head()

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity,circlePolygon
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-73.82254158439093 40.78967994525795...
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.75069910560622 40.72353394538476...
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,"POLYGON ((-73.81550893754734 40.7266959453787,..."
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.78977901254466 40.72528494538140...
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.83001949848953 40.71613994539892...


In [44]:
crashData.to_csv("data/CrashDataWithPolygons-*.csv",index = False)

['/Users/neelgandhi/Big Dat/Final Project/CrashDataWithPolygons.csv/0.part']

In [4]:
crashData = pd.read_csv('data/CrashDataWithPolygons/0.csv')

In [23]:
polyDict = {}
polyList = crashData['circlePolygon'].to_list()
count = 0
for i,pol in enumerate(polyList):
    if pol not in polyDict:
        polyDict[pol] = count
        count += 1

finalList = []
for i in polyList:
    finalList.append(polyDict[i])

crashData['tempID'] = finalList

63643


In [9]:
crashData

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity,circlePolygon,tempID
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-73.82254158439093 40.78967994525795...,0
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.75069910560622 40.72353394538476...,1
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,"POLYGON ((-73.81550893754734 40.7266959453787,...",2
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.78977901254466 40.72528494538140...,3
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.83001949848953 40.71613994539892...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63638,4681109.0,2023,11,20,9,40.607070,-74.092410,STATEN ISLAND,10310,56.099583,5.400000,7.600000,42.30,37.6,46.4,NaN,0.0,13.1,17.1,0.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-74.08886527700406 40.60706994560743...,35731
63639,4681040.0,2023,11,20,9,40.575256,-74.121260,STATEN ISLAND,10310,NaN,4.750000,6.750000,42.30,37.6,46.4,NaN,0.0,13.1,17.1,0.0,0.0,0.0,0.0,0.0,Extreme Severity,"POLYGON ((-74.1177169565362 40.57525594566812,...",26022
63640,4682945.0,2023,11,27,10,40.575813,-74.119790,STATEN ISLAND,10306,NaN,4.750000,6.750000,44.40,41.7,50.0,NaN,0.0,13.6,22.9,0.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-74.11624692715402 40.57581294566705...,38789
63641,4616194.0,2023,3,24,12,40.575985,-74.125680,STATEN ISLAND,10307,NaN,4.750000,6.750000,39.90,35.6,50.7,NaN,0.0,9.0,17.1,0.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-74.12213691808076 40.57598494566673...,26003


In [17]:
# networkData['geometry'] = networkData['geometry'].apply(wkt.loads)

# crashData['circlePolygon'] = crashData['circlePolygon'].apply(wkt.loads)

gdf_points = gpd.GeoDataFrame(networkData, geometry='geometry')
gdf_polygons = gpd.GeoDataFrame(crashData, geometry='circlePolygon')

# Set a common Coordinate Reference System
gdf_points.crs = gdf_polygons.crs = "EPSG:4326"
result = gpd.sjoin(gdf_points, gdf_polygons, how="inner", op='within')

In [18]:
result

,osmid,y,x,street_count,ref,highway,geometry,degree,index_right,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity,tempID
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,12825,4598766.0,2023,1,18,12,40.792362,-73.964180,MANHATTAN,10003,NaN,9.941176,10.176471,40.90,39.0,42.80,38.20,0.000,9.10,15.00,0.0,0.0,0.0,1.0,0.0,Low Severity,10153
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,13690,4609918.0,2023,3,4,0,40.792362,-73.964180,MANHATTAN,10029,NaN,9.941176,10.176471,28.80,21.2,35.60,9.10,0.020,4.30,8.90,0.0,0.0,0.0,1.0,0.0,Low Severity,10153
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,14696,4625504.0,2023,4,21,14,40.790054,-73.965870,MANHATTAN,10003,NaN,9.941176,10.176471,52.60,48.2,55.90,48.10,0.140,8.70,12.00,1.0,0.0,0.0,0.0,0.0,Extreme Severity,11663
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,15977,4638747.0,2023,6,12,16,40.791660,-73.964690,MANHATTAN,10002,NaN,9.941176,10.176471,82.60,69.8,96.80,63.00,0.130,7.40,17.10,0.0,0.0,0.0,0.0,0.0,Extreme Severity,12492
0,30807314,40.790720,-73.963576,4,NaN,NaN,POINT (-73.96358 40.79072),4,16485,4647021.0,2023,7,10,9,40.792180,-73.965950,MANHATTAN,10128,NaN,12.076923,10.076923,76.20,69.1,87.80,59.80,0.070,5.00,15.90,0.0,0.0,0.0,0.0,0.0,Extreme Severity,12792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345958,11858198624,40.727146,-73.853391,4,NaN,NaN,POINT (-73.85339 40.72715),4,43706,4600783.0,2023,1,26,12,40.726864,-73.851990,QUEENS,11358,NaN,5.352941,7.647059,47.70,40.0,55.05,36.15,0.000,8.90,23.55,1.0,0.0,0.0,2.0,0.0,Moderate Severity,18057
345958,11858198624,40.727146,-73.853391,4,NaN,NaN,POINT (-73.85339 40.72715),4,46232,4616038.0,2023,3,27,19,40.728302,-73.854900,QUEENS,11101,NaN,5.929825,7.754386,43.75,39.0,53.95,40.95,0.030,7.55,12.50,1.0,0.0,0.0,1.0,0.0,Low Severity,30304
345958,11858198624,40.727146,-73.853391,4,NaN,NaN,POINT (-73.85339 40.72715),4,49196,4646932.0,2023,7,14,17,40.726570,-73.852646,QUEENS,11377,NaN,5.929825,7.754386,69.70,64.9,93.45,68.05,0.025,8.10,13.00,1.0,0.0,0.0,0.0,0.0,Extreme Severity,18275
345958,11858198624,40.727146,-73.853391,4,NaN,NaN,POINT (-73.85339 40.72715),4,53722,4630952.0,2023,5,19,14,40.726078,-73.851680,QUEENS,11103,NaN,5.929825,7.754386,83.45,72.5,94.45,63.75,0.000,10.50,15.45,0.0,0.0,0.0,0.0,0.0,Extreme Severity,33983


In [28]:
counts = result.groupby(['tempID'])['osmid'].nunique()

In [30]:
countDf = counts.reset_index()
countDf

,tempID,osmid
0,0,76
1,1,147
2,2,249
3,3,75
4,4,182
...,...,...
38784,38785,112
38785,38786,288
38786,38787,55
38787,38788,189


In [31]:
finalData = crashData.merge(countDf, on = ['tempID'])
finalData

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity,circlePolygon,tempID,osmid
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-73.82254158439093 40.78967994525795...,0,76
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.75069910560622 40.72353394538476...,1,147
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,"POLYGON ((-73.81550893754734 40.7266959453787,...",2,249
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.78977901254466 40.72528494538140...,3,75
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-73.83001949848953 40.71613994539892...,4,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63423,4681109.0,2023,11,20,9,40.607070,-74.092410,STATEN ISLAND,10310,56.099583,5.400000,7.600000,42.30,37.6,46.4,NaN,0.0,13.1,17.1,0.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-74.08886527700406 40.60706994560743...,35731,55
63424,4681040.0,2023,11,20,9,40.575256,-74.121260,STATEN ISLAND,10310,NaN,4.750000,6.750000,42.30,37.6,46.4,NaN,0.0,13.1,17.1,0.0,0.0,0.0,0.0,0.0,Extreme Severity,"POLYGON ((-74.1177169565362 40.57525594566812,...",26022,136
63425,4682945.0,2023,11,27,10,40.575813,-74.119790,STATEN ISLAND,10306,NaN,4.750000,6.750000,44.40,41.7,50.0,NaN,0.0,13.6,22.9,0.0,0.0,0.0,0.0,0.0,Extreme Severity,POLYGON ((-74.11624692715402 40.57581294566705...,38789,139
63426,4616194.0,2023,3,24,12,40.575985,-74.125680,STATEN ISLAND,10307,NaN,4.750000,6.750000,39.90,35.6,50.7,NaN,0.0,9.0,17.1,0.0,0.0,0.0,1.0,0.0,Low Severity,POLYGON ((-74.12213691808076 40.57598494566673...,26003,59


In [33]:
finalData = finalData.rename(columns = {'osmid':'numIntersections'})
finalData.drop(columns = ['tempID'], inplace = True)
finalData.to_csv("data/finalCombinedData.csv", index = False)

In [34]:
finalData.nunique()

COLLISION_ID                 63428
Year                             1
Month                           12
Day                             31
hour                            24
LATITUDE                     30223
LONGITUDE                    23789
BOROUGH_x                        5
zipcode                        165
SPEED                         1679
school_count                   369
park_count                     347
MeanTemp                      1431
MinTemp                        755
MaxTemp                        781
DewPoint                      1271
Percipitation                  250
WindSpeed                      515
MaxSustainedWind               172
Rain                             3
SnowDepth                       34
SnowIce                          3
NUMBER OF PERSONS INJURED       20
NUMBER OF PERSONS KILLED         3
Accident Severity                4
circlePolygon                38789
numIntersections               579
dtype: int64